In [37]:
!pip uninstall -y bitsandbytes torch numpy transformers peft datasets accelerate wandb
!pip install torch transformers peft datasets accelerate bitsandbytes
!pip install wandb
!pip install rouge_score
!pip install evaluate
!pip install PyDrive2





  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c67f658ecf6a7acb729c7d0d52d65f961407ca2489140f948d44c5175627647d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [1]:
!git config --global credential.helper store

In [2]:
!huggingface-cli login --token "HF_TOKEN"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `fine-tuning-llama-model` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `fine-tuning-llama-model`


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


model_name = "meta-llama/Llama-3.2-1B"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",  # Ensures proper precision handling
    device_map="auto"  # Uses GPU if available
)

# Check if it works
print("Model loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Model loaded successfully!


In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  # Rank of decomposition
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]  # Apply LoRA to attention layers
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


In [6]:
from datasets import load_dataset

dataset = load_dataset("code-search-net/code_search_net","python")


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [8]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(50000))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(10000))
dataset["validation"] = dataset["validation"].shuffle(seed=42).select(range(20000))

In [9]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token as the padding token

# Tokenization function
def tokenize_function(example):
    tokenized = tokenizer(example["func_code_string"], truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Set labels for loss computation
    return tokenized

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [14]:
columns_to_keep = ["input_ids", "attention_mask"]
tokenized_datasets = tokenized_datasets.remove_columns(
    [col for col in tokenized_datasets["train"].column_names if col not in columns_to_keep]
)


In [10]:
import wandb

# Replace with your actual W&B API key
WANDB_API_KEY = "your_api_token"

# Log in to W&B
wandb.login(key=WANDB_API_KEY)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sagarhv001 (sagarhv001-sagarhv001). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [15]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # Set `mlm=True` for masked LM

# Training Arguments
training_args = TrainingArguments(
    output_dir="./llama-codesearchnet", # Save directory
    run_name="llama-codesearchnet-finetune",  # Set a custom run name
    per_device_train_batch_size=2,  # Reduce batch size to fit in GPU
    gradient_accumulation_steps=4,  # Accumulate gradients for stability
    learning_rate=2e-5,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True,  # Enable mixed precision for better performance
    push_to_hub=False, # Disable Hugging Face Hub push
    remove_unused_columns=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)




<ipython-input-15-d3a8d08bd71c>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

Step,Training Loss
500,7.340700
1000,7.137900
1500,7.159500
2000,7.120200
2500,7.084700
3000,7.091400
3500,7.077800
4000,7.123400
4500,7.047900
5000,7.077400


TrainOutput(global_step=18750, training_loss=7.064436282552084, metrics={'train_runtime': 28901.1705, 'train_samples_per_second': 5.19, 'train_steps_per_second': 0.649, 'total_flos': 4.48818315264e+17, 'train_loss': 7.064436282552084, 'epoch': 3.0})

In [17]:
metrics = trainer.evaluate()
print(metrics)

{'eval_runtime': 1818.2336, 'eval_samples_per_second': 11.0, 'eval_steps_per_second': 1.375, 'epoch': 3.0}


In [25]:
test_code = "Write a Python code to implement a function to take a list input of numbers as int and str, i.e [1,2,3,'4',5] and change it to ['1','2','3',4,'5']"

inputs = tokenizer(test_code, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_length=100)
generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Code:\n", generated_code)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Code:
 Write a Python code to implement a function to take a list input of numbers as int and str, i.e [1,2,3,'4',5] and change it to ['1','2','3',4,'5'].
```
def change_list_to_str(my_list):
    new_list = []
    for item in my_list:
        if type(item) is int:
            new_list.append(str(item))
        else:
            new_list.append(item)
    return new_list


In [30]:
import json
import torch

# Save model
model.save_pretrained("/kaggle/working/llama_finetuned_model")
tokenizer.save_pretrained("/kaggle/working/llama_finetuned_model")

('/kaggle/working/llama_finetuned_model/tokenizer_config.json',
 '/kaggle/working/llama_finetuned_model/special_tokens_map.json',
 '/kaggle/working/llama_finetuned_model/tokenizer.json')